In [5]:
import pandas as pd
import numpy as np
import ast

In [6]:
movies = pd.read_csv(r'C:\Users\paula\OneDrive\Curso\Henry\Proyecto Individual\Movies\movies_dataset.csv')
credits = pd.read_csv(r'C:\Users\paula\OneDrive\Curso\Henry\Proyecto Individual\Movies\credits.csv')

C:\Users\paula\AppData\Local\Temp\ipykernel_18824\724676636.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(r'C:\Users\paula\OneDrive\Curso\Henry\Proyecto Individual\Movies\movies_dataset.csv')


In [7]:
movies[['revenue', 'budget']] = movies[['revenue', 'budget']].fillna(0)

In [8]:
# Asegurar que 'revenue' y 'budget' son numéricos y reemplazar nulos por 0
movies['revenue'] = pd.to_numeric(movies['revenue'], errors='coerce').fillna(0)
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce').fillna(0)

# Calcular 'return'
movies['return'] = np.where(movies['budget'] == 0, 0, movies['revenue'] / movies['budget'])

# Reemplazar divisiones por cero o valores nulos directamente
movies['return'] = movies['return'].replace([np.inf, np.nan], 0)


In [9]:
# Convertir release_date al formato datetime y manejar errores
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')

# Crear la columna release_year extrayendo el año
movies['release_year'] = movies['release_date'].dt.year

# Crear la columna release_month extrayendo el año
movies['release_month'] = movies['release_date'].dt.month

# Crear la columna release_dow extrayendo el año
movies['release_dow'] = movies['release_date'].dt.day_of_week

# Eliminar aquellos registros que quedaron nulos
movies = movies[movies['release_year'].notnull()]

In [10]:
movies = movies.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage', 'spoken_languages', 'production_countries', 'belongs_to_collection', 'genres', 'production_companies', 'original_language','tagline', 'overview', 'runtime', 'status'], errors='ignore')


In [11]:
movies.head()

,budget,id,popularity,release_date,revenue,title,vote_average,vote_count,return,release_year,release_month,release_dow
0,30000000.0,862,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,12.451801,1995.0,10.0,0.0
1,65000000.0,8844,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,4.043035,1995.0,12.0,4.0
2,0.0,15602,11.7129,1995-12-22,0.0,Grumpier Old Men,6.5,92.0,0.000000,1995.0,12.0,4.0
3,16000000.0,31357,3.859495,1995-12-22,81452156.0,Waiting to Exhale,6.1,34.0,5.090760,1995.0,12.0,4.0
4,0.0,11862,8.387519,1995-02-10,76578911.0,Father of the Bride Part II,5.7,173.0,0.000000,1995.0,2.0,4.0


In [12]:
movies['popularity'] = pd.to_numeric(movies['popularity'], errors='coerce')


In [ ]:
# Guardar el dataset transformado
movies.to_csv(r'Datasets/transformed_movies.csv', index=False)

In [19]:
# Convertir la columna `crew` en listas de diccionarios
credits['crew'] = credits['crew'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extraer el nombre de los directores
credits['director_name'] = credits['crew'].apply(
    lambda x: next((person['name'] for person in x if person.get('job') == 'Director'), None)
)

# Asegurarse de que la columna `cast` se convierta en listas de diccionarios
credits['cast'] = credits['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extraer los nombres de todos los actores
credits['actors'] = credits['cast'].apply(
    lambda x: [actor['name'] for actor in x] if isinstance(x, list) else []
)

# Crear un nuevo DataFrame reducido
reduced_credits = credits[['id', 'director_name', 'actors']]

# Guardar el dataset reducido
reduced_credits.to_csv(r'Datasets/reduced_credits1.csv', index=False)